In [5]:
# We download our dataset for the training

!pip3 install gutenbergpy
import datasource
datasource.download_dataset()

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Cache already exists
0 / 28
1 / 28
2 / 28
3 / 28
4 / 28
5 / 28
6 / 28
7 / 28
8 / 28
9 / 28
10 / 28
11 / 28
12 / 28
13 / 28
14 / 28
15 / 28
16 / 28
17 / 28
18 / 28
19 / 28
20 / 28
21 / 28
22 / 28
23 / 28
24 / 28
25 / 28
26 / 28
27 / 28


In [5]:
!pip3 install keras_nlp
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization
import tensorflow_text as tf_text
import keras_nlp
import numpy as np
import pickle
import random
from pathlib import Path
from vectorization import nw_split

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [6]:
APP_DIR = Path.home() / ".lafontaine"
DATASET = APP_DIR / "dataset.txt"
VECTORIZER = APP_DIR / "vectorizer.pkl"

In [7]:
with DATASET.open('r') as stream:
    verses = stream.read().splitlines()
    verses = verses[0:10000]    
    verses = [' '.join(v.split(' ')[::-1]) for v in verses]
    
print(len(verses))   
print(verses[0])

    
# We force it at 50 or it won't fit in GPU memory
seq_length = len(max(verses, key=lambda v: len(v.split())))
# seq_length = 50

vectorizer = TextVectorization(
    split=nw_split,
    standardize='lower',
    output_mode="int",
    output_sequence_length=seq_length + 1,
)

vectorizer.adapt(verses)
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
index_lookup = dict(zip(range(len(vocab)), vocab))

with VECTORIZER.open('wb') as stream:
    pickle.dump(
        {
            'config': vectorizer.get_config(),
            'weights': vectorizer.get_weights(),
            'seq_length': seq_length
        }, 
        stream
    )

index_lookup

10000
sa à accoudée rêve qui femme jeune La


{0: '',
 1: '[UNK]',
 2: 'et',
 3: 'de',
 4: 'la',
 5: 'les',
 6: 'le',
 7: 'des',
 8: 'un',
 9: 'en',
 10: 'je',
 11: 'à',
 12: 'dans',
 13: 'qui',
 14: 'que',
 15: 'du',
 16: 'il',
 17: 'au',
 18: 'sur',
 19: 'est',
 20: 'comme',
 21: 'se',
 22: 'une',
 23: 'son',
 24: 'plus',
 25: 'ce',
 26: 'pour',
 27: 'ses',
 28: 'où',
 29: 'ne',
 30: 'mon',
 31: 'avec',
 32: 'l',
 33: 'par',
 34: 'sa',
 35: 'a',
 36: 'pas',
 37: 'on',
 38: 'aux',
 39: 'mais',
 40: 'leurs',
 41: 'ma',
 42: 'leur',
 43: 'vers',
 44: 'si',
 45: 'nous',
 46: 'vous',
 47: 'tout',
 48: 'elle',
 49: 'sont',
 50: 'sans',
 51: 'me',
 52: 'tu',
 53: 'moi',
 54: 'd',
 55: "d'un",
 56: 'cœur',
 57: "c'est",
 58: 'sous',
 59: 'mes',
 60: 'ou',
 61: 'yeux',
 62: 'mort',
 63: 'ces',
 64: 'ciel',
 65: 'là',
 66: 'lui',
 67: 'ils',
 68: 'dont',
 69: 'suis',
 70: "j'ai",
 71: 'soir',
 72: 'ont',
 73: 'tous',
 74: 'fait',
 75: 's',
 76: 'qu',
 77: 'ton',
 78: 'temps',
 79: 'jour',
 80: 'n',
 81: 'même',
 82: 'quand',
 83: 'toujour

In [8]:
random.shuffle(verses)
length = len(verses)
text_train = verses[:int(0.7*length)]
text_test = verses[int(0.7*length):int(0.85*length)]
text_valid = verses[int(0.85*length):]

batch_size = 32

train_dataset = tf.data.Dataset.from_tensor_slices(text_train)
train_dataset = train_dataset.shuffle(buffer_size=256)
train_dataset = train_dataset.batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices(text_test)
test_dataset = test_dataset.shuffle(buffer_size=256)
test_dataset = test_dataset.batch(batch_size)

valid_dataset = tf.data.Dataset.from_tensor_slices(text_valid)
valid_dataset = valid_dataset.shuffle(buffer_size=256)
valid_dataset = valid_dataset.batch(batch_size)

def preprocess_text(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorizer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y

train_dataset = train_dataset.map(preprocess_text)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

test_dataset = test_dataset.map(preprocess_text)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

valid_dataset = valid_dataset.map(preprocess_text)
valid_dataset = valid_dataset.prefetch(tf.data.AUTOTUNE)

In [9]:
embed_dim = 128
num_heads = 4

def create_model():
    inputs = keras.layers.Input(shape=(seq_length,), dtype=tf.int32)
    embedding_layer = keras_nlp.layers.TokenAndPositionEmbedding(vocab_size, seq_length, embed_dim)(inputs)
    decoder = keras_nlp.layers.TransformerDecoder(intermediate_dim=embed_dim, 
                                                            num_heads=num_heads, 
                                                            dropout=0.5)(embedding_layer)
    
    outputs = keras.layers.Dense(vocab_size, activation='softmax')(decoder)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer="adam", 
        loss='sparse_categorical_crossentropy',
        metrics=[keras_nlp.metrics.Perplexity(), 'accuracy']
    )
    return model

model = create_model()
model.summary()

/usr/local/lib/python3.8/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 34)]              0         
                                                                 
 token_and_position_embeddin  (None, 34, 128)          1466624   
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_decoder (Transf  (None, 34, 128)          99584     
 ormerDecoder)                                                   
                                                                 
 dense (Dense)               (None, 34, 11424)         1473696   
                                                                 
Total params: 3,039,904
Trainable params: 3,039,904
Non-trainable params: 0
___________________________________________________

In [10]:
class TextSampler(keras.callbacks.Callback):
    def __init__(self, start_prompt, max_tokens):
        self.start_prompt = start_prompt
        self.max_tokens = max_tokens
        
    # Helper method to choose a word from the top K probable words with respect to their probabilities
    # in a sequence
    def sample_token(self, logits):
        logits, indices = tf.math.top_k(logits, k=5, sorted=True)
        indices = np.asarray(indices).astype("int32")
        preds = keras.activations.softmax(tf.expand_dims(logits, 0))[0]
        preds = np.asarray(preds).astype("float32")
        return np.random.choice(indices, p=preds)

    def on_epoch_end(self, epoch, logs=None):
        decoded_sample = self.start_prompt
        
        for i in range(self.max_tokens-1):
            tokenized_prompt = vectorizer([decoded_sample])[:, :-1]
            predictions = self.model.predict([tokenized_prompt], verbose=0)
            # To find the index of the next word in the prediction array.
            # The tokenized prompt is already shorter than the original decoded sample
            # by one, len(decoded_sample.split()) is two words ahead - so we remove 1 to get
            # the next word in the sequence
            sample_index = len(decoded_sample.strip().split())-1
            
            sampled_token = self.sample_token(predictions[0][sample_index])
            sampled_token = index_lookup[sampled_token]
            decoded_sample += " " + sampled_token
            
        print(f"\nSample text:\n{decoded_sample}...\n")

# First 5 words of a random sentence to be used as a seed
random_sentence = ' '.join(random.choice(text_valid).replace('\n', ' ').split(' ')[:4])

# Sampler from seed
sampler = TextSampler(random_sentence, 30)

# Reduce learning rate when a metric has stopped improving.
reducelr = keras.callbacks.ReduceLROnPlateau(patience=10, monitor='val_loss')

In [11]:
model = create_model()
history = model.fit(train_dataset, 
                    validation_data=valid_dataset,
                    epochs=10, 
                    callbacks=[sampler, reducelr])
# Save model
model.save("sentence_model")
# model = keras.models.load_model("sentence_model", compile=False)

Epoch 1/10
219/219 [==============================] - ETA: 0s - loss: 2.0770 - perplexity: 7.9807 - accuracy: 0.8258
Sample text:
mous, rosiers les parmi   de le   un de  je et  le  et  et des et   de  le  et  la ...

219/219 [==============================] - 12s 47ms/step - loss: 2.0770 - perplexity: 7.9807 - accuracy: 0.8258 - val_loss: 1.1631 - val_perplexity: 3.1997 - val_accuracy: 0.8398 - lr: 0.0010
Epoch 2/10
219/219 [==============================] - ETA: 0s - loss: 1.0644 - perplexity: 2.8991 - accuracy: 0.8470
Sample text:
mous, rosiers les parmi l il que tandis qui je qui    et  de l  mais de et mais  et  et  que  de comme ...

219/219 [==============================] - 10s 45ms/step - loss: 1.0644 - perplexity: 2.8991 - accuracy: 0.8470 - val_loss: 1.1008 - val_perplexity: 3.0066 - val_accuracy: 0.8494 - lr: 0.0010
Epoch 3/10
219/219 [==============================] - ETA: 0s - loss: 0.9205 - perplexity: 2.5105 - accuracy: 0.8604
Sample text:
mous, rosiers les parmi où et 

INFO:tensorflow:Assets written to: sentence_model/assets


INFO:tensorflow:Assets written to: sentence_model/assets


In [12]:
def sample_token(logits):
        logits, indices = tf.math.top_k(logits, k=5, sorted=True)
        indices = np.asarray(indices).astype("int32")
        preds = keras.activations.softmax(tf.expand_dims(logits, 0))[0]
        preds = np.asarray(preds).astype("float32")
        return np.random.choice(indices, p=preds)

def generate_text(prompt, response_length=20):
    decoded_sample = prompt
    for i in range(response_length-1):
        tokenized_prompt = vectorizer([decoded_sample])[:, :-1]
        predictions = model.predict([tokenized_prompt], verbose=0)
        sample_index = len(decoded_sample.strip().split())-1
        sampled_token = sample_token(predictions[0][sample_index])
        sampled_token = index_lookup[sampled_token]
        decoded_sample += " " + sampled_token
    return decoded_sample

In [13]:
generate_text("Nous")

'Nous pour demande lui caractères les que  et  invente je  et  mère sa de que parce'